In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
iris=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

In [6]:
iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [9]:
mnist=pd.read_csv('https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/mnist_test.csv',header=None)

In [10]:
mnist.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**IRIS**

Let’s identify a portion of our data for which we will try to guess the species. Randomly select 20%
of the records and store the first four columns (i.e. the features representing each flower) into a
two-dimensional numpy array of shape N × C, you can call it X_test.
For the same records, store the last column (i.e. the one with the species values) into another array,
namely y_test. This is the data that will be used to test the accuracy of your KNN implementation
and its correct functioning (i.e. the testing data).

In [13]:
iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [28]:
values=iris.values
X=iris.iloc[:,:4]
y=iris[4]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) #split the data

In [33]:
X_train.shape

(120, 4)

In [ ]:
class KNearestNeighbors:
    

    def __init__(self, k, distance_metric="euclidean"):
        self.k = k
        self.distance_metric = distance_metric
        self.X=None
        self.y=None

    def fit(self, X, y):
        from sklearn.preprocessing import MinMaxScaler #We need this to normalize the data
        X_size = X.shape
        #mm=MinMaxScaler()
        #self.X,self.y=mm.fit_transform(X,y)
        self.X=X
        self.y=y
        return self

        
    def predict(self, X):
        if self.distance_metric=="euclidean":
            from scipy.spatial.distance import euclidean as d
        elif self.distance_metric== "cosine":
            from scipy.spatial.distance import cosine as d
        else:
            from scipy.spatial.distance import cityblock as d
        
        distance=d(X,self.X)
        d_m=min(distance)
        position=distance.index(d_m)
        flower=self.y[position]
        return flower
        """Run the KNN classification on X.
        :param X: input data points, ndarray, shape = (N,C).
        :return: labels : ndarray, shape = (N,).
        """
        pass # TODO: implement it!



In [37]:
s = "math"
from s import sqrt


ModuleNotFoundError: No module named 's'

In [34]:
def ed(p,q,n):
    sum=0
    for i in range(n):
        sum+=(p[i]-q[i])**2
    distance=np.sqrt(sum)
    return distance

In [35]:
def cd(p,q,n):
    sumq=0
    sump=0
    sumpq=0
    for i in range(n):
        sumq+=q[i]**2
        sump+=p[i]**2
        sumpq+=p[i]*q[i]
    cs=sumpq/(sump*sumq)
    return cs

In [36]:
def md(p,q,n):
    md=0
    for i in range(n):
        md+=np.abs(p[i]-q[i])
    return md